# Style-Bert-VITS2 (ver 2.7.0) のGoogle Colabでの学習

Google Colab上でStyle-Bert-VITS2の学習を行うことができます。

このnotebookでは、通常使用ではあなたのGoogle Driveにフォルダ`Style-Bert-VITS2`を作り、その内部での作業を行います。他のフォルダには触れません。
Google Driveを使わない場合は、初期設定のところで適切なパスを指定してください。

## 流れ

### 学習を最初からやりたいとき
上から順に実行していけばいいです。音声合成に必要なファイルはGoogle Driveの`Style-Bert-VITS2/model_assets/`に保存されます。また、途中経過も`Style-Bert-VITS2/Data/`に保存されるので、学習を中断したり、途中から再開することもできます。

### 学習を途中から再開したいとき
0と1を行い、3の前処理は飛ばして、4から始めてください。スタイル分け5は、学習が終わったら必要なら行ってください。


## 0. 環境構築

Style-Bert-VITS2の環境をcolab上に構築します。ランタイムがT4等のGPUバックエンドになっていることを確認し、実行してください。

**注意**: このセルを実行した後に「セッションがクラッシュしました」「不明な理由により、セッションがクラッシュしました。」等の警告が出ますが、**無視してそのまま先へ**進んでください。（一度ランタイムを再起動させてnumpy<2を強制させるため `exit()` を呼んでいることからの措置です。）

In [13]:
import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements-colab.txt --no-progress
!python initialize.py --skip_default_models

exit()

downloading uv 0.8.14 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Cloning into 'Style-Bert-VITS2'...
remote: Enumerating objects: 6852, done.
remote: Counting objects: 100% (3817/3817), done.
remote: Compressing objects: 100% (1066/1066), done.
remote: Total 6852 (delta 2909), reused 2771 (delta 2751), pack-reused 3035 (from 1)
Receiving objects: 100% (6852/6852), 14.58 MiB | 22.77 MiB/s, done.
Resolving deltas: 100% (4605/4605), done.
/content/Style-Bert-VITS2/Style-Bert-VITS2
Using Python 3.12.11 environment at: /usr
Audited 26 packages in 300ms
08-31 07:15:20 |  INFO  | initialize.py:19 | Downloading deberta-v2-large-japanese-char-wwm pytorch_model.bin
pytorch_model.bin: 100% 1.32G/1.32G [00:07<00:00, 169MB/s]
08-31 07:15:28 |  INFO  | initialize.py:19 | Downloading deberta-v2-large-japanese-char-wwm-onnx model_fp16.onnx
model_fp16.onnx: 100% 653M/653M [00:05<00:00, 129MB/s]
08-31 07:15:33 |  INFO  | initialize.py:1

In [3]:
# Google driveを使う方はこちらを実行してください。

from google.colab import drive

drive.mount("/content/drive")

ValueError: Mountpoint must not already contain files

## 1. 初期設定

学習とその結果を保存するディレクトリ名を指定します。
Google driveの場合はそのまま実行、カスタマイズしたい方は変更して実行してください。

In [4]:
# 作業ディレクトリを移動
%cd /content/Style-Bert-VITS2/

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = "/content/drive/MyDrive/Style-Bert-VITS2/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = "/content/drive/MyDrive/Style-Bert-VITS2/model_assets"

import yaml


with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

/content/Style-Bert-VITS2


## 2. 学習に使うデータ準備

すでに音声ファイル（1ファイル2-12秒程度）とその書き起こしデータがある場合は2.2を、ない場合は2.1を実行してください。

### 2.1 音声ファイルからのデータセットの作成（ある人はスキップ可）

音声ファイル（1ファイル2-12秒程度）とその書き起こしのデータセットを持っていない方は、（日本語の）音声ファイルのみから以下の手順でデータセットを作成することができます。Google drive上の`Style-Bert-VITS2/inputs/`フォルダに音声ファイル（wavやmp3等の通常の音声ファイル形式、1ファイルでも複数ファイルでも可）を置いて、下を実行すると、データセットが作られ、自動的に正しい場所へ配置されます。

**2024-06-02のVer 2.5以降**、`inputs/`フォルダにサブフォルダを2個以上作ってそこへ音声ファイルをスタイルに応じて振り分けて置くと、学習の際にサブディレクトリに応じたスタイルが自動的に作成されます。デフォルトスタイルのみでよい場合や手動でスタイルを後で作成する場合は`inputs/`直下へ入れれば大丈夫です。

In [18]:
# 元となる音声ファイル（wav形式）を入れるディレクトリ
input_dir = "/content/drive/MyDrive/Style-Bert-VITS2/inputs"
# モデル名（話者名）を入力
model_name = "your_model_name"

# こういうふうに書き起こして欲しいという例文（句読点の入れ方・笑い方や固有名詞等）
initial_prompt = "こんにちは。元気、ですかー？ふふっ、私は……ちゃんと元気だよ！"

!python slice.py -i {input_dir} --model_name {model_name}
!python transcribe.py --model_name {model_name} --initial_prompt {initial_prompt} --use_hf_whisper

08-31 07:54:33 |  INFO  | slice.py:167 | Found 1 audio files.
08-31 07:54:33 |WARNING | slice.py:169 | Output directory /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/raw already exists, deleting...
Using cache found in /root/.cache/torch/hub/litagin02_silero-vad_master
  0%|          | 0/1 [00:00<?, ?it/s]Using cache found in /root/.cache/torch/hub/litagin02_silero-vad_master
/root/.cache/torch/hub/litagin02_silero-vad_master/utils_vad.py:127: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  audio_backends = torchaudio.list_audio_backends()
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:2

In [17]:
!python transcribe.py --model_name {model_name} --initial_prompt {initial_prompt} --use_hf_whisper --hf_repo_id "openai/whisper-large-v3"

08-31 07:52:44 |  INFO  | transcribe.py:157 | Found 1 WAV files
08-31 07:52:44 |  INFO  | transcribe.py:204 | Loading HF Whisper model (openai/whisper-large-v3)
  0%|          | 0/1 [00:00<?, ?it/s]2025-08-31 07:52:47.861515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756626767.882547   20400 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756626767.889860   20400 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756626767.906254   20400 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756626767.906282   20400 computation_placer.cc:177]

成功したらそのまま3へ進んでください

### 2.2 音声ファイルと書き起こしデータがすでにある場合

指示に従って適切にデータセットを配置してください。

次のセルを実行して、学習データをいれるフォルダ（1で設定した`dataset_root`）を作成します。

In [ ]:
import os

os.makedirs(dataset_root, exist_ok=True)

まず音声データと、書き起こしテキストを用意してください。

それを次のように配置します。
```
├── Data/
│   ├── {モデルの名前}
│   │   ├── esd.list
│   │   ├── raw/
│   │   │   ├── foo.wav
│   │   │   ├── bar.mp3
│   │   │   ├── style1/
│   │   │   │   ├── baz.wav
│   │   │   │   ├── qux.wav
│   │   │   ├── style2/
│   │   │   │   ├── corge.wav
│   │   │   │   ├── grault.wav
...
```

### 配置の仕方
- 上のように配置すると、`style1/`と`style2/`フォルダの内部（直下以外も含む）に入っている音声ファイルたちから、自動的にデフォルトスタイルに加えて`style1`と`style2`というスタイルが作成されます
- 特にスタイルを作る必要がない場合や、スタイル分類機能等でスタイルを作る場合は、`raw/`フォルダ直下に全てを配置してください。このように`raw/`のサブディレクトリの個数が0または1の場合は、スタイルはデフォルトスタイルのみが作成されます。
- 音声ファイルのフォーマットはwav形式以外にもmp3等の多くの音声ファイルに対応しています

### 書き起こしファイル`esd.list`

`Data/{モデルの名前}/esd.list` ファイルには、以下のフォーマットで各音声ファイルの情報を記述してください。


```
path/to/audio.wav(wavファイル以外でもこう書く)|{話者名}|{言語ID、ZHかJPかEN}|{書き起こしテキスト}
```

- ここで、最初の`path/to/audio.wav`は、`raw/`からの相対パスです。つまり、`raw/foo.wav`の場合は`foo.wav`、`raw/style1/bar.wav`の場合は`style1/bar.wav`となります。
- 拡張子がwavでない場合でも、`esd.list`には`wav`と書いてください、つまり、`raw/bar.mp3`の場合でも`bar.wav`と書いてください。


例：
```
foo.wav|hanako|JP|こんにちは、元気ですか？
bar.wav|taro|JP|はい、聞こえています……。何か用ですか？
style1/baz.wav|hanako|JP|今日はいい天気ですね。
style1/qux.wav|taro|JP|はい、そうですね。
...
english_teacher.wav|Mary|EN|How are you? I'm fine, thank you, and you?
...
```
もちろん日本語話者の単一話者データセットでも構いません。

## 3. 学習の前処理

次に学習の前処理を行います。必要なパラメータをここで指定します。次のセルに設定等を入力して実行してください。「～～かどうか」は`True`もしくは`False`を指定してください。

In [28]:
# 上でつけたフォルダの名前`Data/{model_name}/`
model_name = "your_model_name"

# JP-Extra （日本語特化版）を使うかどうか。日本語の能力が向上する代わりに英語と中国語は使えなくなります。
use_jp_extra = True

# 学習のバッチサイズ。VRAMのはみ出具合に応じて調整してください。
batch_size = 4

# 学習のエポック数（データセットを合計何周するか）。
# 100で多すぎるほどかもしれませんが、もっと多くやると質が上がるのかもしれません。
epochs = 100

# 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000

# 音声ファイルの音量を正規化するかどうか
normalize = False

# 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = False

# 読みのエラーが出た場合にどうするか。
# "raise"ならテキスト前処理が終わったら中断、"skip"なら読めない行は学習に使わない、"use"なら無理やり使う
yomi_error = "skip"

上のセルが実行されたら、次のセルを実行して学習の前処理を行います。

In [29]:
from gradio_tabs.train import preprocess_all
from style_bert_vits2.nlp.japanese import pyopenjtalk_worker


pyopenjtalk_worker.initialize_worker()

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    num_processes=2,
    normalize=normalize,
    trim=trim,
    freeze_EN_bert=False,
    freeze_JP_bert=False,
    freeze_ZH_bert=False,
    freeze_style=False,
    freeze_decoder=False,
    use_jp_extra=use_jp_extra,
    val_per_lang=0,
    log_interval=200,
    yomi_error=yomi_error,
)

08-31 08:15:03 |  INFO  | train.py:72 | Step 1: start initialization...
model_name: your_model_name, batch_size: 4, epochs: 100, save_every_steps: 1000, freeze_ZH_bert: False, freeze_JP_bert: False, freeze_EN_bert: False, freeze_style: False, freeze_decoder: False, use_jp_extra: True
08-31 08:15:03 |WARNING | train.py:103 | Step 1: /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/models already exists, so copy it to backup to /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/models_backup
08-31 08:15:22 |SUCCESS | train.py:132 | Step 1: initialization finished.
08-31 08:15:22 |  INFO  | train.py:137 | Step 2: start resampling...
08-31 08:15:22 |  INFO  | subprocess.py:23 | Running: resample.py -i /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/raw -o /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/wavs --num_processes 2 --sr 44100
08-31 08:15:27 |SUCCESS | subprocess.py:38 | Success: resample.py -i /content/drive/MyDrive/Style-Bert-VITS2

(True, 'Success: 全ての前処理が完了しました。ターミナルを確認しておかしいところがないか確認するのをおすすめします。')

## 4. 学習

前処理が正常に終わったら、学習を行います。次のセルを実行すると学習が始まります。

学習の結果は、上で指定した`save_every_steps`の間隔で、Google Driveの中の`Style-Bert-VITS2/Data/{モデルの名前}/model_assets/`フォルダに保存されます。

このフォルダをダウンロードし、ローカルのStyle-Bert-VITS2の`model_assets`フォルダに上書きすれば、学習結果を使うことができます。

In [6]:
# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
model_name = "your_model_name"


import yaml
from gradio_tabs.train import get_path

paths = get_path(model_name)
dataset_path = str(paths.dataset_path)
config_path = str(paths.config_path)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

ModuleNotFoundError: No module named 'gradio_tabs'

In [31]:
# 日本語特化版を「使う」場合
!python train_ms_jp_extra.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

2025-08-31 08:17:36.391591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756628256.412058   26909 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756628256.418136   26909 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756628256.433678   26909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756628256.433703   26909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756628256.433708   26909 computation_placer.cc:177] computation placer alr

In [ ]:
# 日本語特化版を「使わない」場合
!python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

In [11]:
# 学習結果を試す・マージ・スタイル分けはこちらから
!python app.py --share

2025-08-31 07:24:22.259310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756625062.279522   13119 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756625062.285635   13119 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756625062.301315   13119 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756625062.301344   13119 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756625062.301348   13119 computation_placer.cc:177] computation placer alr

In [ ]:
# ONNX変換は、変換したいsafetensorsファイルを指定してこのセルを実行してください。
!python convert_onnx.py --model "Data/your_model/your_model_e100_s10000.safetensors"

In [16]:
!cat /content/Style-Bert-VITS2/transcribe.py

import argparse
import sys
from pathlib import Path
from typing import Any, Optional

from torch.utils.data import Dataset
from tqdm import tqdm

from config import get_path_config
from style_bert_vits2.constants import Languages
from style_bert_vits2.logging import logger
from style_bert_vits2.utils.stdout_wrapper import SAFE_STDOUT


# faster-whisperは並列処理しても速度が向上しないので、単一モデルでループ処理する
def transcribe_with_faster_whisper(
    model: "WhisperModel",
    audio_file: Path,
    initial_prompt: Optional[str] = None,
    language: str = "ja",
    num_beams: int = 1,
    no_repeat_ngram_size: int = 10,
):
    segments, _ = model.transcribe(
        str(audio_file),
        beam_size=num_beams,
        language=language,
        initial_prompt=initial_prompt,
        no_repeat_ngram_size=no_repeat_ngram_size,
    )
    texts = [segment.text for segment in segments]
    return "".join(texts)


# HF pipelineで進捗表示をするために必要なDatasetクラス
class StrListDataset(Dataset[str]):
    def __init__(self, original

In [19]:
import os

dataset_root = "/content/drive/MyDrive/Style-Bert-VITS2/Data"
model_name = "your_model_name"
esd_list_path = os.path.join(dataset_root, model_name, "esd.list")

if os.path.exists(esd_list_path):
    print(f"'{esd_list_path}' が見つかりました。")
else:
    print(f"'{esd_list_path}' は見つかりませんでした。")
    print("データ準備のステップが正しく完了しているか再度確認してください。")

'/content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/esd.list' は見つかりませんでした。
データ準備のステップが正しく完了しているか再度確認してください。


In [22]:
import os
from pathlib import Path
from style_bert_vits2.logging import logger
from transformers import WhisperProcessor, pipeline
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
from typing import Optional, Any

# HF pipelineで進捗表示をするために必要なDatasetクラス
class StrListDataset(Dataset[str]):
    def __init__(self, original_list: list[str]) -> None:
        self.original_list = original_list

    def __len__(self) -> int:
        return len(self.original_list)

    def __getitem__(self, i: int) -> str:
        return self.original_list[i]

# HFのWhisperはファイルリストを与えるとバッチ処理ができて速い
def transcribe_files_with_hf_whisper(
    audio_files: list[Path],
    model_id: str,
    output_file: Path,
    initial_prompt: Optional[str] = None,
    language: str = "ja",
    batch_size: int = 16,
    num_beams: int = 1,
    no_repeat_ngram_size: int = 10,
    device: str = "cuda",
    pbar: Optional[tqdm] = None,
) -> list[str]:
    logger.info(f"Loading HF Whisper model ({model_id})")

    processor: WhisperProcessor = WhisperProcessor.from_pretrained(model_id)
    generate_kwargs: dict[str, Any] = {
        "language": language,
        "do_sample": False,
        "num_beams": num_beams, # Corrected syntax: num_beams=...
        "no_repeat_ngram_size": no_repeat_ngram_size, # Corrected syntax: no_repeat_ngram_size=...
    }
    logger.info(f"generate_kwargs: {generate_kwargs}, loading pipeline...")
    pipe = pipeline(
        model=model_id,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=batch_size,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device=device,
        trust_remote_code=True,
        # generate_kwargs=generate_kwargs, # NOTE: generate_kwargs is passed below in the pipe call
    )
    logger.info("Loaded pipeline")
    if initial_prompt is not None:
        prompt_ids: torch.Tensor = pipe.tokenizer.get_prompt_ids(
            initial_prompt, return_tensors="pt"
        ).to(device)
        generate_kwargs["prompt_ids"] = prompt_ids


    dataset = StrListDataset([str(f) for f in audio_files])

    results: list[str] = []
    for whisper_result, file in zip(
        pipe(dataset, generate_kwargs=generate_kwargs), audio_files
    ):
        text: str = whisper_result["text"]
        # なぜかテキストの最初に" {initial_prompt}"が入るので、文字の最初からこれを削除する
        # cf. https://github.com/huggingface/transformers/issues/27594
        if initial_prompt is not None and text.startswith(f" {initial_prompt}"):
            text = text[len(f" {initial_prompt}") :]

        # Write to esd.list
        try:
            # Use relative path from dataset_root/model_name/raw/
            input_dir = output_file.parent / "raw"
            wav_rel_path = file.relative_to(input_dir)
            language_id = "JP" # Assuming JP based on previous setup, needs adjustment if language is variable

            with open(output_file, "a", encoding="utf-8") as f:
                f.write(f"{wav_rel_path}|{model_name}|{language_id}|{text.strip()}\n") # Added strip() to remove leading/trailing whitespace
            results.append(text)
            if pbar is not None:
                pbar.update(1)
        except Exception as e:
            logger.error(f"Error writing to esd.list for file {file}: {e}")
            # Continue processing other files even if one fails to write


    if pbar is not None:
        pbar.close()

    return results

# --- Execution starts here ---
dataset_root = Path("/content/drive/MyDrive/Style-Bert-VITS2/Data")
model_name = "your_model_name" # Make sure this matches your model name
input_dir = dataset_root / model_name / "raw"
output_file = dataset_root / model_name / "esd.list"
initial_prompt = "こんにちは。元気、ですかー？ふふっ、私は……ちゃんと元気だよ！" # Make sure this matches your initial prompt
language = "ja"
model_id = "openai/whisper-large-v3" # Use the working model ID
batch_size = 16
num_beams = 1
no_repeat_ngram_size = 10
device = "cuda"

# Ensure output directory exists
output_file.parent.mkdir(parents=True, exist_ok=True)

# Find WAV files in the raw directory
wav_files = [f for f in input_dir.rglob("*.wav") if f.is_file()]
wav_files = sorted(wav_files, key=lambda x: str(x))

if len(wav_files) == 0:
    print(f"No WAV files found in {input_dir}. Please ensure slice.py ran correctly.")
else:
    print(f"Found {len(wav_files)} WAV files in {input_dir}.")
    # Delete esd.list if it exists to avoid appending to old data
    if output_file.exists():
        logger.warning(f"{output_file} exists, backing up to {output_file}.bak")
        backup_path = output_file.with_name(output_file.name + ".bak")
        if backup_path.exists():
             backup_path.unlink() # Delete existing backup
        output_file.rename(backup_path)
        logger.info(f"Backed up existing {output_file} to {backup_path}")


    pbar = tqdm(total=len(wav_files), dynamic_ncols=True)

    transcribe_files_with_hf_whisper(
        audio_files=wav_files,
        model_id=model_id,
        output_file=output_file,
        initial_prompt=initial_prompt,
        language=language,
        batch_size=batch_size,
        num_beams=num_beams,
        no_repeat_ngram_size=no_repeat_ngram_size,
        device=device,
        pbar=pbar,
    )

    print(f"\nTranscription process completed. Checking if {output_file} was created.")

    # Final check after transcription
    if output_file.exists():
        print(f"'{output_file}' が見つかりました。")
        # Optional: Display content of esd.list
        # with open(output_file, 'r', encoding='utf-8') as f:
        #     print("\nContent of esd.list:")
        #     print(f.read())
    else:
        print(f"'{output_file}' は見つかりませんでした。")
        print("書き起こしファイル生成に失敗した可能性があります。")

Found 1 WAV files in /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/raw.


  0%|          | 0/1 [00:00<?, ?it/s]

08-31 07:58:37 |  INFO  | ipython-input-4168086732.py:34 | Loading HF Whisper model (openai/whisper-large-v3)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


08-31 07:58:39 |  INFO  | ipython-input-4168086732.py:43 | generate_kwargs: {'language': 'ja', 'do_sample': False, 'num_beams': 1, 'no_repeat_ngram_size': 10}, loading pipeline...


Device set to use cuda
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


08-31 07:58:49 |  INFO  | ipython-input-4168086732.py:54 | Loaded pipeline


100%|██████████| 1/1 [00:15<00:00, 15.02s/it]


Transcription process completed. Checking if /content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/esd.list was created.
'/content/drive/MyDrive/Style-Bert-VITS2/Data/your_model_name/esd.list' が見つかりました。


In [32]:
import os

model_assets_path = "/content/drive/MyDrive/Style-Bert-VITS2/model_assets/your_model_name"
model_file_name = "your_model_name_e100_s100.safetensors"
model_file_path = os.path.join(model_assets_path, model_file_name)

if os.path.exists(model_file_path):
    print(f"'{model_file_path}' が見つかりました。")
else:
    print(f"'{model_file_path}' は見つかりませんでした。")
    print("学習が正常に完了し、モデルファイルが保存されているか確認してください。")

'/content/drive/MyDrive/Style-Bert-VITS2/model_assets/your_model_name/your_model_name_e100_s100.safetensors' が見つかりました。


In [4]:
import yaml

config_path = "/content/Style-Bert-VITS2/config.yml"

with open(config_path, "r", encoding="utf-8") as f:
    config_data = yaml.safe_load(f)

# Update the model path for the webui section
# Assuming the model is saved as your_model_name/your_model_name_e100_s100.safetensors
# relative to the assets_root
model_name = config_data.get("model_name", "your_model_name") # Get model_name from config, default to your_model_name
model_file_name = f"{model_name}_e100_s100.safetensors" # Construct the expected file name
config_data["webui"]["model"] = f"{model_name}/{model_file_name}" # Set the path relative to assets_root


with open(config_path, "w", encoding="utf-8") as f:
    yaml.dump(config_data, f, allow_unicode=True)

print(f"'{config_path}' を更新しました。 webui.model のパスを '{config_data['webui']['model']}' に設定しました。")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Style-Bert-VITS2/config.yml'

`config.yml` の更新が完了したら、再度以下のコードセルを実行して `app.py` を起動してください。今度は学習したモデルが読み込まれるはずです。

In [2]:
# 学習結果を試す・マージ・スタイル分けはこちらから
!python app.py --share

python3: can't open file '/content/app.py': [Errno 2] No such file or directory
